In [49]:
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F, Compose
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
import json
import os
import random
from torch.optim.lr_scheduler import StepLR
import numpy as np

In [50]:
class CocoTransform:
    def __init__(self, min_size=640, max_size=800):
        self.min_size = min_size
        self.max_size = max_size
    
    def __call__(self, image, target):
        
        w, h = image.size
        scale = min(self.min_size / min(w, h), self.max_size / max(w, h))
        new_w = int(w * scale)
        new_h = int(h * scale)
        
        image = F.resize(image, [new_h, new_w])
        image = F.to_tensor(image)
        # Normalize with ImageNet stats if needed
        # image = F.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        image = F.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        if target and "boxes" in target and len(target["boxes"]) > 0:
            # Scale bounding boxes
            boxes = target["boxes"].clone()
            boxes[:, [0, 2]] = boxes[:, [0, 2]] * scale
            boxes[:, [1, 3]] = boxes[:, [1, 3]] * scale
            
            # Enforce valid coordinates with more conservative clamping
            boxes[:, 0] = torch.clamp(boxes[:, 0], min=0, max=new_w - 2)
            boxes[:, 1] = torch.clamp(boxes[:, 1], min=0, max=new_h - 2)
            boxes[:, 2] = torch.clamp(boxes[:, 2], min=boxes[:, 0] + 1, max=new_w)
            boxes[:, 3] = torch.clamp(boxes[:, 3], min=boxes[:, 1] + 1, max=new_h)
            
            valid_boxes = (boxes[:, 2] > boxes[:, 0] + 1) & (boxes[:, 3] > boxes[:, 1] + 1)

            if torch.sum(valid_boxes) > 0:
                boxes = boxes[valid_boxes]
                if "labels" in target:
                    target["labels"] = target["labels"][valid_boxes]
                if "area" in target:
                    # Recalculate areas after transformation
                    target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
                if "iscrowd" in target:
                    target["iscrowd"] = target["iscrowd"][valid_boxes]

            target["boxes"] = boxes
            # Recalculate areas after transformation
            # target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        return image, target

In [51]:
class CocoDataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None, max_samples=None):
        super().__init__(img_folder, ann_file)
        self._transforms = transforms
        self.coco = COCO(ann_file)
        
        # Option to limit dataset size
        if max_samples is not None and max_samples < len(self.ids):
            self.ids = self.ids[:max_samples]
        
        # Map category ids to continuous indices starting from 1 (0 is background)
        self.cat_ids = sorted(self.coco.getCatIds())
        self.cat_id_to_continuous = {cat_id: i + 1 for i, cat_id in enumerate(self.cat_ids)}

    def __getitem__(self, idx):
        # Safety counter to prevent infinite recursion
        safety_counter = 0
        max_attempts = 10
        original_idx = idx

        while safety_counter < max_attempts:
            try:
                img, anns = super().__getitem__(idx)
                image_id = self.ids[idx]
                
                boxes = []
                labels = []
                
                for ann in anns:
                    if ann.get('iscrowd', 0):
                        continue
                    
                    try:
                        x, y, w, h = ann['bbox']
                    except (KeyError, ValueError) as e:
                        continue  # Skip annotations with invalid bbox format
                    
                    # More aggressive filtering of small boxes
                    if w < 3 or h < 3:
                        continue

                    x, y, w, h = float(x), float(y), float(w), float(h)
                    if not (w > 0 and h > 0 and x >= 0 and y >= 0):
                        continue

                    boxes.append([x, y, x + w, y + h])
                    
                    # Safely get category ID
                    category_id = ann.get('category_id')
                    if category_id is None or category_id not in self.cat_id_to_continuous:
                        continue
                        
                    # Map category ID to continuous index
                    labels.append(self.cat_id_to_continuous[category_id])

                if len(boxes) > 0:
                    try:
                        boxes = torch.tensor(boxes, dtype=torch.float32)
                        labels = torch.tensor(labels, dtype=torch.int64)
                        image_id = torch.tensor([image_id])
                        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
                        
                        # More aggressive filtering
                        valid_indices = (area > 9.0)  # Minimum area of 3x3
                        if torch.sum(valid_indices) == 0:
                            idx = (idx + 1) % len(self)
                            safety_counter += 1
                            continue
                            
                        boxes = boxes[valid_indices]
                        labels = labels[valid_indices]
                        area = area[valid_indices]
                        
                        # Double-check we still have boxes
                        if len(boxes) == 0:
                            idx = (idx + 1) % len(self)
                            safety_counter += 1
                            continue
                            
                        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

                        target = {
                            "boxes": boxes, 
                            "labels": labels, 
                            "image_id": image_id,
                            "area": area,
                            "iscrowd": iscrowd
                        }
                        
                        if self._transforms:
                            img, target = self._transforms(img, target)
                            
                            # Skip samples with no valid boxes after transforms
                            if "boxes" not in target or len(target["boxes"]) == 0:
                                idx = (idx + 1) % len(self)
                                safety_counter += 1
                                continue
                                
                            # Final validity check
                            if torch.any(torch.isnan(target["boxes"])) or \
                               torch.any(target["boxes"][:, 2] <= target["boxes"][:, 0]) or \
                               torch.any(target["boxes"][:, 3] <= target["boxes"][:, 1]):
                                idx = (idx + 1) % len(self)
                                safety_counter += 1
                                continue
                            
                        return img, target
                    except Exception as e:
                        print(f"Error processing boxes for image idx {idx}: {e}")
                        idx = (idx + 1) % len(self)
                        safety_counter += 1
                        continue
                
                # If no valid boxes, try the next image
                idx = (idx + 1) % len(self)
                safety_counter += 1
                
            except Exception as e:
                print(f"Error processing image idx {idx}: {e}")
                idx = (idx + 1) % len(self)
                safety_counter += 1
                
        # If we get here, we couldn't find a good sample after max_attempts
        print(f"Warning: Could not find valid annotations after {max_attempts} attempts starting from idx {original_idx}")
        
        # If we couldn't find a good image after max attempts, create a dummy example
        # with a single small box to avoid errors
        dummy_box = torch.tensor([[10, 10, 20, 20]], dtype=torch.float32)
        dummy_label = torch.tensor([1], dtype=torch.int64)
        dummy_target = {
            "boxes": dummy_box,
            "labels": dummy_label,
            "image_id": torch.tensor([self.ids[idx]]),
            "area": torch.tensor([100.0]),
            "iscrowd": torch.tensor([0], dtype=torch.int64)
        }
        
        # Create a default image if the original one failed
        try:
            if self._transforms:
                img, dummy_target = self._transforms(img, dummy_target)
        except:
            # Create a simple black image as fallback
            img = torch.zeros((3, 640, 640), dtype=torch.float32)
            # Apply ImageNet normalization
            img[0] = -0.485 / 0.229  # Normalized black for R channel
            img[1] = -0.456 / 0.224  # Normalized black for G channel
            img[2] = -0.406 / 0.225  # Normalized black for B channel
        
        print(f"Warning: Could not find valid annotations after {max_attempts} attempts")
        return img, dummy_target

In [52]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [53]:
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, classifications, targets):
        # Flatten the batch
        batch_size = classifications.shape[0]
        classifications = classifications.view(-1)
        targets = targets.view(-1)
        
        # Binary cross entropy
        BCE = torch.nn.functional.binary_cross_entropy_with_logits(
            classifications, targets.float(), reduction='none'
        )
        
        # Apply focal loss formula
        pt = torch.exp(-BCE)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE
        
        return F_loss.sum() / batch_size

In [54]:
def get_model(num_classes):
    # Create backbone
    backbone = resnet_fpn_backbone('resnet50', pretrained=True)
    
    # Freeze backbone layers
    for name, param in backbone.named_parameters():
        if 'layer4' not in name:
            param.requires_grad = False
    
    # Create FasterRCNN model with reduced parameters for GTX 1650
    model = FasterRCNN(
        backbone=backbone,
        num_classes=num_classes,
        rpn_pre_nms_top_n_train=500,  # Reduced from 2000
        rpn_pre_nms_top_n_test=250,    # Reduced from 1000
        rpn_post_nms_top_n_train=500, # Reduced from 2000
        rpn_post_nms_top_n_test=250,   # Reduced from 1000
        rpn_score_thresh=0.05,
        box_score_thresh=0.05,
        box_nms_thresh=0.5
    )
    
    return model

In [55]:
def evaluate_model(model, data_loader, device, ann_file):
    model.eval()
    coco_results = []
    
    for images, targets in tqdm(data_loader, desc="Evaluating"):
        images = [img.to(device) for img in images]
        
        with torch.no_grad():
            with torch.cuda.amp.autocast(enabled=True):
                outputs = model(images)
        
        for target, output in zip(targets, outputs):
            image_id = int(target["image_id"].item())
            boxes = output["boxes"].cpu().numpy()
            scores = output["scores"].cpu().numpy()
            labels = output["labels"].cpu().numpy()
            
            for box, score, label in zip(boxes, scores, labels):
                if score < 0.05:  # Skip low confidence predictions
                    continue
                    
                x_min, y_min, x_max, y_max = box
                width, height = x_max - x_min, y_max - y_min
                
                # Convert back from continuous index to original category id
                dataset = data_loader.dataset
                original_cat_id = dataset.cat_ids[label - 1] if label - 1 < len(dataset.cat_ids) else label
                
                coco_results.append({
                    "image_id": image_id,
                    "category_id": int(original_cat_id),
                    "bbox": [float(x_min), float(y_min), float(width), float(height)],
                    "score": float(score)
                })
    
    # Save predictions
    pred_file = "coco_predictions.json"
    with open(pred_file, "w") as f:
        json.dump(coco_results, f)
    
    # Evaluate with COCO API
    coco_gt = COCO(ann_file)
    coco_dt = coco_gt.loadRes(pred_file)
    
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    
    metrics = {
        "mAP": coco_eval.stats[0],  # mAP@[0.5:0.95]
        "AP50": coco_eval.stats[1],  # AP@0.50
        "AP75": coco_eval.stats[2],  # AP@0.75
        "AP_small": coco_eval.stats[3],  # AP for small objects
        "AP_medium": coco_eval.stats[4],  # AP for medium objects
        "AP_large": coco_eval.stats[5],  # AP for large objects
        "AR": coco_eval.stats[8]  # AR@100
    }
    
    return metrics

In [56]:
def train():

    seed = 42
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False
    
    # Set multiprocessing method to "spawn" to avoid issues (mainly for Windows)
    try:
        import multiprocessing
        multiprocessing.set_start_method('spawn')
    except RuntimeError:
        pass

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Set paths - update these to your COCO dataset paths
    data_root = "D:/coco_dataset"
    train_img_folder = f"{data_root}/train2017/train2017"
    train_ann_file = f"{data_root}/annotations_trainval2017/annotations/instances_train2017.json"
    val_img_folder = f"{data_root}/val2017/val2017"
    val_ann_file = f"{data_root}/annotations_trainval2017/annotations/instances_val2017.json"
    
    # Create datasets with aspect-ratio preserving transforms (smaller size for GTX 1650)
    transform = CocoTransform(min_size=640, max_size=800)
    
    print("Loading training dataset...")
    train_dataset = CocoDataset(
        img_folder=train_img_folder,
        ann_file=train_ann_file,
        transforms=transform,
        max_samples=2500  # Further limited for GTX 1650
    )
    
    print("Loading validation dataset...")
    val_dataset = CocoDataset(
        img_folder=val_img_folder,
        ann_file=val_ann_file,
        transforms=transform,
        max_samples=500  # Further limited for GTX 1650
    )
    
    # Number of classes (80 COCO classes + background)
    num_classes = len(train_dataset.cat_ids) + 1
    print(f"Number of classes: {num_classes}")
    
    # Create data loaders
    batch_size = 2  # Reduced for GTX 1650 with 4GB VRAM
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        collate_fn=collate_fn, 
        num_workers=0,  # Reduced for lower memory usage
        pin_memory=True,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        collate_fn=collate_fn, 
        num_workers=0,  # Reduced for lower memory usage
        pin_memory=True
    )
    
    # Create model
    print("Creating model...")
    model = get_model(num_classes).to(device)
    
    # Set up optimizer and learning rate scheduler
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=0.00001, weight_decay=0.0001)
    lr_scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
    
    # Set up gradient scaler for mixed precision training
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    
    clip_value = 0.1

    # Training loop - reduced epochs for GTX 1650
    num_epochs = 3
    best_map = 0.0
    patience = 0
    max_patience = 3

    # Memory management for GTX 1650
    torch.cuda.empty_cache()
    
    print("Starting training...")
    start_epoch = 0
    checkpoint_path = "latest_checkpoint.pth"
    if os.path.exists(checkpoint_path):
        try:
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            print(f"Resuming from epoch {start_epoch}")
        except Exception as e:
            print(f"Failed to load checkpoint: {e}")
    
    for epoch in range(start_epoch, num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        loss_count = 0
        
        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            try:
                # Skip batches with no targets
                if any(len(t["boxes"]) == 0 for t in targets):
                    print("Skipping batch with empty targets")
                    continue
                
                # Move data to device
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                
                # Zero gradients
                optimizer.zero_grad()
                
                # Forward pass with mixed precision
                with torch.cuda.amp.autocast(enabled=True):
                    loss_dict = model(images, targets)
                    
                    # Check for NaN values in individual losses
                    has_nan = False
                    for k, v in loss_dict.items():
                        if torch.isnan(v):
                            print(f"NaN detected in {k}")
                            has_nan = True
                    
                    if has_nan:
                        print("Warning: NaN loss detected! Skipping batch.")
                        continue
                    
                    losses = sum(loss for loss in loss_dict.values())
    
                    if torch.isnan(losses) or losses == 0:
                        print("Warning: NaN in total loss! Skipping batch.")
                        continue

                # Backward pass with gradient scaling
                scaler.scale(losses).backward()
                
                # Unscale before clipping to ensure proper scaling
                scaler.unscale_(optimizer)
                
                # Clip gradients to prevent explosion
                torch.nn.utils.clip_grad_norm_(params, clip_value)
                
                # Check for NaN gradients after backward
                has_nan_grad = False
                for param in params:
                    if param.grad is not None and torch.isnan(param.grad).any():
                        has_nan_grad = True
                        break
                
                if has_nan_grad:
                    print("Warning: NaN gradients detected! Skipping optimizer step.")
                    optimizer.zero_grad()
                    continue

                scaler.step(optimizer)
                scaler.update()
                
                total_loss += losses.item()
                loss_count += 1
                
                # Print more frequent loss updates
                if loss_count % 50 == 0:
                    print(f"  Batch {loss_count}, Current loss: {losses.item():.4f}")
                    
                # Add gradient debugging - print norm occasionally
                if loss_count % 100 == 0:
                    total_norm = 0
                    for p in params:
                        if p.grad is not None:
                            param_norm = p.grad.detach().data.norm(2)
                            total_norm += param_norm.item() ** 2
                    total_norm = total_norm ** 0.5
                    print(f"  Gradient norm: {total_norm:.4f}")
                
            except Exception as e:
                print(f"Error during training: {e}")
                continue
        
        if loss_count > 0:
            avg_loss = total_loss / loss_count
            print(f"Epoch {epoch+1}/{num_epochs}, Avg Loss: {avg_loss:.4f}")
        else:
            print(f"Epoch {epoch+1}/{num_epochs}, No valid batches!")
        
        # Update learning rate
        lr_scheduler.step()
        
        # Save checkpoint every epoch (for recovery)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': lr_scheduler.state_dict(),
        }, f"checkpoint_epoch_{epoch+1}.pth")
        
        # Validation phase 
        print("Running validation...")
        try:
            metrics = evaluate_model(model, val_loader, device, val_ann_file)
            
            print(f"Validation metrics:")
            print(f"  mAP@[0.5:0.95]: {metrics['mAP']:.4f}")
            print(f"  AP@0.50: {metrics['AP50']:.4f}")
            print(f"  AP@0.75: {metrics['AP75']:.4f}")
            
            # Early stopping check
            if metrics['mAP'] > best_map:
                best_map = metrics['mAP']
                patience = 0
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': lr_scheduler.state_dict(),
                    'mAP': best_map,
                }, "best_fasterrcnn_coco.pth")
                print(f"New best model saved with mAP: {best_map:.4f}")
            else:
                patience += 1
                print(f"No improvement for {patience} epochs")
                
            if patience >= max_patience:
                print(f"Early stopping after {patience} epochs without improvement")
                break
                
        except Exception as e:
            print(f"Error during validation: {e}")
    
    # Final evaluation and save
    print("Final evaluation...")
    try:
        metrics = evaluate_model(model, val_loader, device, val_ann_file)
        
        print(f"Final metrics:")
        print(f"  mAP@[0.5:0.95]: {metrics['mAP']:.4f}")
        print(f"  AP@0.50: {metrics['AP50']:.4f}")
        print(f"  AP@0.75: {metrics['AP75']:.4f}")
        
        # Save final model
        torch.save({
            'model_state_dict': model.state_dict(),
            'num_classes': num_classes,
            'cat_id_mapping': train_dataset.cat_id_to_continuous,
            'metrics': metrics
        }, "final_fasterrcnn_coco.pth")
        print("Final model saved to final_fasterrcnn_coco.pth")
    except Exception as e:
        print(f"Error during final evaluation: {e}")
        # Save anyway
        torch.save({
            'model_state_dict': model.state_dict(),
            'num_classes': num_classes,
            'cat_id_mapping': train_dataset.cat_id_to_continuous,
        }, "final_fasterrcnn_coco.pth")
        print("Final model saved (without metrics) to final_fasterrcnn_coco.pth")

In [57]:
def inference(image_path, model_path, device=None):
    """
    Run inference on a single image using a saved model
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load model checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    
    # Create model with same architecture
    num_classes = checkpoint.get('num_classes', 91)
    model = get_model(num_classes).to(device)
    
    # Load weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Load and transform image
    from PIL import Image
    image = Image.open(image_path).convert("RGB")
    
    # Apply transforms - using smaller sizes for GTX 1650
    transform = CocoTransform(min_size=640, max_size=800)
    image_tensor, _ = transform(image, {})
    
    # Run inference with memory optimization
    torch.cuda.empty_cache()  # Clear GPU memory before inference
    with torch.no_grad():
        with torch.cuda.amp.autocast(enabled=True):
            prediction = model([image_tensor.to(device)])
    
    return prediction[0], image

In [58]:
if __name__ == "__main__":
    import platform
    if platform.system() == 'Windows':

        import os
        os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    train()

Using device: cuda
Loading training dataset...
loading annotations into memory...
Done (t=27.49s)
creating index...
index created!
loading annotations into memory...
Done (t=24.68s)
creating index...
index created!
Loading validation dataset...
loading annotations into memory...
Done (t=0.74s)
creating index...
index created!
loading annotations into memory...
Done (t=0.64s)
creating index...
index created!
Number of classes: 81
Creating model...


C:\Users\DHAIRYA\AppData\Local\Temp\ipykernel_17164\2335064246.py:86: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Starting training...


Epoch 1/3 - Training:   0%|          | 0/1250 [00:00<?, ?it/s]

Error processing boxes for image idx 1219: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 1220: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 1221: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 1222: clamp() received an inva

...

Error during training: unscale_() has already been called on this optimizer since the last update().
Error processing boxes for image idx 373: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 374: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 375: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max 

Epoch 1/3 - Training:   2%|▏         | 21/1250 [00:51<55:58,  2.73s/it]

Error during training: unscale_() has already been called on this optimizer since the last update().
Error processing boxes for image idx 374: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 375: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)

Error processing boxes for image idx 376: clamp() received an invalid combination of arguments - got (Tensor, min=Tensor, max=int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max 

Epoch 1/3 - Training:   2%|▏         | 21/1250 [00:53<51:49,  2.53s/it]


KeyboardInterrupt: 